# Language modelling
The exercise shows how a language model may be used to solve word-prediction tasks and to generate text.

## Download three Polish models from the Huggingface repository.

In [1]:
!pip install transformers
!pip install torch


     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 55.3 MB/s 
     |████████████████████████████████| 61 kB 504 kB/s 
     |████████████████████████████████| 3.3 MB 68.4 MB/s 
     |████████████████████████████████| 895 kB 76.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

In [9]:
herbert_tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
herbert_model = AutoModelForMaskedLM.from_pretrained("allegro/herbert-base-cased")

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
large_tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-large-cased")
large_model = AutoModelForMaskedLM.from_pretrained("allegro/herbert-large-cased")

Some weights of the model checkpoint at allegro/herbert-large-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
bert_tokenizer = AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")
bert_model = AutoModelForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
def print_top5_unmask(tokenizer, model, sequence):
  sequence = sequence.replace('[MASK]', tokenizer.mask_token)
  inputs = tokenizer(sequence, return_tensors="pt")
  mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
  token_logits = model(**inputs).logits
  mask_token_logits = token_logits[0, mask_token_index, :]
  top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
  for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

## Produce the predictions for the following sentences (use each model and check 5 predictions)

In [7]:
sequences = [
             "Warszawa to największe [MASK].",
             "Te zabawki należą do [MASK].",
             "Policjant przygląda się [MASK].",
             "Na środku skrzyżowania widać [MASK].",
             "Właściciel samochodu widział złodzieja z [MASK].",
             "Prezydent z premierem rozmawiali wczoraj o [MASK].",
             "Witaj drogi [MASK]."
]

In [8]:
class Model:
  def __init__(self, name, tokenizer, model):
    self.name = name
    self.tokenizer = tokenizer
    self.model = model

In [12]:
models = [
          Model('herbert-base-cased', herbert_tokenizer, herbert_model), 
          Model('herbert-large-cased', large_tokenizer, large_model),
          Model('bert-base-polish-cased-v1', bert_tokenizer, bert_model)
          ]

In [18]:
for model in models:
  print('-'*100)
  print(model.name)
  print()
  for sequence in sequences:
    print_top5_unmask(model.tokenizer, model.model, sequence)
    print()

----------------------------------------------------------------------------------------------------
herbert-base-cased

Warszawa to największe miasto.
Warszawa to największe lotnisko.
Warszawa to największe centrum.
Warszawa to największe miasta.
Warszawa to największe atrakcje.

Te zabawki należą do rodziny.
Te zabawki należą do nas.
Te zabawki należą do nich.
Te zabawki należą do najlepszych.
Te zabawki należą do ..

Policjant przygląda się mężczyźnie.
Policjant przygląda się kobiecie.
Policjant przygląda się mu.
Policjant przygląda się dziewczynie.
Policjant przygląda się sprawie.

Na środku skrzyżowania widać rondo.
Na środku skrzyżowania widać samochody.
Na środku skrzyżowania widać radiowóz.
Na środku skrzyżowania widać samochód.
Na środku skrzyżowania widać wiadukt.

Właściciel samochodu widział złodzieja z samochodu.
Właściciel samochodu widział złodzieja z włamaniem.
Właściciel samochodu widział złodzieja z auta.
Właściciel samochodu widział złodzieja z kierowcą.
Właściciel s

## Check the model predictions for the following sentences (using each model).

In [19]:
sequences = [
             "Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].",
             "Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK]."
]

In [20]:
for model in models:
  print('-'*100)
  print(model.name)
  print()
  for sequence in sequences:
    print_top5_unmask(model.tokenizer, model.model, sequence)
    print()

----------------------------------------------------------------------------------------------------
herbert-base-cased

Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie poddał.
Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zdziwił.
Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie dowiedział.
Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zastanawiał.
Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie przyznał.

Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie dowiedziała.
Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie przyznała.
Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie bała.
Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zmieniła.
Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zgodziła.

----------------------------------------------------------------------------------------------------
herbert-l

## Check the model predictions for the following sentences.

In [21]:
sequences = [
             "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.",
             "W wakacje odwiedziłem [MASK], który jest stolicą Islandii.",
             "Informatyka na [MASK] należy do najlepszych kierunków w Polsce."
]

In [22]:
for model in models:
  print('-'*100)
  print(model.name)
  print()
  for sequence in sequences:
    print_top5_unmask(model.tokenizer, model.model, sequence)
    print()

----------------------------------------------------------------------------------------------------
herbert-base-cased

Woda wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Słońce wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Ziemia wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Następnie wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Ciało wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.

W wakacje odwiedziłem Kraków, który jest stolicą Islandii.
W wakacje odwiedziłem Oslo, który jest stolicą Islandii.
W wakacje odwiedziłem Londyn, który jest stolicą Islandii.
W wakacje odwiedziłem Gdańsk, który jest stolicą Islandii.
W wakacje odwiedziłem Toruń, który jest stolicą Islandii.

Informatyka na pewno należy do najlepszych kierunków w Polsce.
Informatyka na AGH należy do najlepszych kierunków w Polsce.
Informatyka na UW należy do najlepszych k

Answer the following questions:

    

1.   Which of the models produced the best results? *Herbert-large-cased*
2.   Was any of the models able to capture Polish grammar? *All*
3.   Was any of the models able to capture long-distant relationships between the words? *Herbert-large-cased*
4.   Was any of the models able to capture world knowledge? *All (water)*
5.   What are the most striking errors made by the models? "Warszawa to największe miasta." or replacing [MASK] with "." 


Wykonał **Mikołaj Sikora**

Grupa 12:50 Piątek